## 学术资源加速
方便从 huggingface 下载模型，这是云平台 autodl 提供的，仅适用于 autodl

In [ ]:
import subprocess
import os

result = subprocess.run('bash -c "source /etc/network_turbo && env | grep proxy"', shell=True, capture_output=True, text=True)
output = result.stdout
for line in output.splitlines():
    if '=' in line:
        var, value = line.split('=', 1)
        os.environ[var] = value

## 步骤 1 导入相关包
开始之前，我们需要导入适用于模型训练和推理的必要库，如 transformers。

In [ ]:
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForSeq2Seq, TrainingArguments, Trainer

## 步骤 2 加载数据集
使用适当的数据加载器，例如 datasets 库，来加载预处理过的指令遵循性任务数据集。

In [ ]:
ds = Dataset.load_from_disk("/root/PEFT代码/tuning/lesson01/data/alpaca_data_zh/")
ds

## 步骤 3 数据集预处理

利用预训练模型的分词器（Tokenizer）对原始文本进行编码，并生成相应的输入 ID、注意力掩码和标签。

### 1）获取分词器

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("Langboat/bloom-1b4-zh")
tokenizer

### 2）定义数据处理函数

In [ ]:
def process_func(example):
    MAX_LENGTH = 256
    input_ids, attention_mask, labels = [], [], []
    instruction = tokenizer("\n".join(["Human: " + example["instruction"], example["input"]]).strip() + "\n\nAssistant: ")
    response = tokenizer(example["output"] + tokenizer.eos_token)
    input_ids = instruction["input_ids"] + response["input_ids"]
    attention_mask = instruction["attention_mask"] + response["attention_mask"]
    labels = [-100] * len(instruction["input_ids"]) + response["input_ids"]
    if len(input_ids) > MAX_LENGTH:
        input_ids = input_ids[:MAX_LENGTH]
        attention_mask = attention_mask[:MAX_LENGTH]
        labels = labels[:MAX_LENGTH]

    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels
    }


### 3）对数据进行预处理

In [ ]:
tokenized_ds = ds.map(process_func, remove_columns=ds.column_names)
tokenized_ds

## 步骤 4 创建模型
然后，我们实例化一个预训练模型，这个模型将作为微调的基础。对于大型模型，我们可能还需要进行一些特定的配置，以适应可用的计算资源。


In [ ]:
model = AutoModelForCausalLM.from_pretrained("Langboat/bloom-1b4-zh", low_cpu_mem_usage=True)

下面2个部分是Prompt Tuning相关的配置。
### 1、PEFT 步骤 1 配置文件
在使用 PEFT 进行微调时，我们首先需要创建一个配置文件，该文件定义了微调过程中的各种设置，如学习率调度、优化器选择等。
提前安装 peft：pip install peft

In [ ]:
from peft import PromptTuningConfig, get_peft_model, TaskType, PromptTuningInit

config = PromptTuningConfig(task_type=TaskType.CAUSAL_LM, num_virtual_tokens=10)
config

### 2、PEFT 步骤 2 创建模型
接下来，我们使用 PEFT 和预训练模型来创建一个微调模型。这个模型将包含原始的预训练模型以及由 PEFT 引入的低秩参数。

In [ ]:
model = get_peft_model(model, config)
model

## 步骤 5 配置训练参数
在这一步，我们定义训练参数，这些参数包括输出目录、学习率、权重衰减、梯度累积步数、训练周期数等。这些参数将被用来配置训练过程。

In [ ]:
args = TrainingArguments(
    output_dir="/root/autodl-tmp/tuningdata/prompt_tuning", 
    per_device_train_batch_size=4, 
    gradient_accumulation_steps=8, 
    logging_steps=10,  
    num_train_epochs=1  
)

## 步骤 6 创建训练器
最后，我们创建一个训练器实例，它封装了训练循环。训练器将负责运行训练过程，并根据我们之前定义的参数进行优化。

In [ ]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_ds,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True),
)

## 步骤 7 模型训练
通过调用训练器的train()方法，我们启动模型的训练过程。这将根据之前定义的参数执行模型的训练。

In [ ]:
trainer.train()

## 步骤 8 模型推理
训练完成后，我们可以使用训练好的模型进行推理。

In [ ]:
from peft import PeftModel
from transformers import pipeline


model = AutoModelForCausalLM.from_pretrained("Langboat/bloom-1b4-zh", low_cpu_mem_usage=True)
tokenizer = AutoTokenizer.from_pretrained("Langboat/bloom-1b4-zh")


p_model = PeftModel.from_pretrained(model=model, model_id="/root/autodl-tmp/tuningdata/prompt_tuning/checkpoint-500")


pipe = pipeline("text-generation", model=p_model, tokenizer=tokenizer, device=0)
ipt = "Human: {}\n{}".format("如何写好一个简历？", "").strip() + "\n\nAssistant: "
pipe(ipt, max_length=256, do_sample=True, )
